# Training

Experiments were conducted using:
- `mb_vae_dti/training/run.py`: command line interface for training
- and the scripts in `scripts/training/` for running the experiments

This notebook shows some plots and analysis of the unsupervised pretraining, general DTI training and benchmark fine-tuning results.

In [49]:
from resolve import *

In [50]:
from mb_vae_dti.validating.analysis import *

df = load_gridsearch_results("notebooks/results/summer/version3/baseline_davis_cold")

if "metadata.data.provenance_cols" in df.columns:
    df["metadata.data.provenance_cols"] = df["metadata.data.provenance_cols"].apply(lambda x: x[0])
if "metadata.data.drug_features" in df.columns:
    df = df.drop(columns=["metadata.data.drug_features", "metadata.data.target_features"])

df

2025-07-27 16:26:08,796 - mb_vae_dti.validating.analysis - INFO - Successfully loaded 486 results with 37 columns


,experiment_name,best_val_loss,best_epoch,trainable_params,timestamp,metadata.training.max_epochs,metadata.training.early_stopping_patience,metadata.training.scheduler,metadata.training.learning_rate,metadata.data.load_in_memory,...,metrics.val/ci,metrics.val/mse,metrics.val/pearson,metrics.val/r2,metrics.val/rmse,timing.total_training_time,timing.avg_time_per_epoch,timing.total_epochs,timing.min_epoch_time,timing.max_epoch_time
0,baseline_DAVIS_cold_b03c0019_1753060088,0.692918,12,4226816,1.753060e+09,100,10,one_cycle,0.0010,True,...,0.616333,1.024365,0.270351,-0.296472,1.167569,249.318542,10.839937,23,10.656162,10.761292
1,baseline_DAVIS_cold_b00c0047_1753066006,0.675439,21,1491072,1.753066e+09,100,10,one_cycle,0.0020,True,...,0.674276,0.779529,0.375212,0.013408,1.072749,101.951710,3.185991,32,2.872527,3.342249
2,baseline_DAVIS_cold_b01c0050_1753065556,0.657510,19,2050560,1.753066e+09,100,10,one_cycle,0.0020,True,...,0.677624,0.709509,0.379269,0.102030,0.998819,175.011104,5.833703,30,5.451108,5.952737
3,baseline_DAVIS_cold_b02c0093_1753078747,0.803100,17,12114944,1.753079e+09,100,10,one_cycle,0.0005,True,...,0.577102,0.950694,0.203909,-0.203222,1.320383,69.117996,2.468500,28,2.263101,2.676257
4,baseline_DAVIS_cold_b01c0102_1753076287,0.717081,10,21043200,1.753076e+09,100,10,one_cycle,0.0020,True,...,0.653011,0.830799,0.329922,-0.051480,1.216416,61.303524,2.919215,21,2.761841,2.938515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,baseline_DAVIS_cold_b03c0034_1753063491,0.603203,22,1588736,1.753064e+09,100,10,one_cycle,0.0005,True,...,0.637636,1.185985,0.307570,-0.501008,1.201909,149.742441,4.537650,33,4.206006,4.692100
482,baseline_DAVIS_cold_b01c0073_1753070898,0.668279,15,22105088,1.753071e+09,100,10,one_cycle,0.0020,True,...,0.665559,0.697174,0.349462,0.117632,1.007518,435.718640,16.758409,26,16.456089,16.734961
483,baseline_DAVIS_cold_b02c0115_1753084023,0.669082,29,1838272,1.753085e+09,100,10,one_cycle,0.0010,True,...,0.660525,0.689010,0.368374,0.127965,1.011807,517.264223,12.931606,40,12.727803,12.904892
484,baseline_DAVIS_cold_b03c0085_1753074756,0.754580,11,12645888,1.753075e+09,100,10,one_cycle,0.0020,True,...,0.630070,1.030333,0.302875,-0.304012,1.047667,136.835160,6.219780,22,6.055933,6.121187


In [51]:
varying_attrs = get_varying_attributes(df)
varying_attrs.remove('metadata.logging.experiment_name')
varying_attrs.extend(["best_val_loss", 'best_epoch', 'trainable_params', "timing.total_training_time"])
varying_attrs

2025-07-27 16:26:10,473 - mb_vae_dti.validating.analysis - INFO - Found 7 varying attributes: ['metadata.training.learning_rate', 'metadata.data.batch_size', 'metadata.logging.experiment_name', 'metadata.model.encoder_kwargs.hidden_dim', 'metadata.model.encoder_kwargs.n_layers', 'metadata.model.embedding_dim', 'metadata.model.encoder_type']


['metadata.training.learning_rate',
 'metadata.data.batch_size',
 'metadata.model.encoder_kwargs.hidden_dim',
 'metadata.model.encoder_kwargs.n_layers',
 'metadata.model.embedding_dim',
 'metadata.model.encoder_type',
 'best_val_loss',
 'best_epoch',
 'trainable_params',
 'timing.total_training_time']

In [52]:
subdf = get_top_performers(df, n_top=150)[varying_attrs]
# for each columns, rename the column by splitting the column name by "." and taking the last element
subdf.columns = subdf.columns.str.split(".").str[-1]
print(df["timing.total_training_time"].mean())
subdf

216.64715845153165


,learning_rate,batch_size,hidden_dim,n_layers,embedding_dim,encoder_type,best_val_loss,best_epoch,trainable_params,total_training_time
308,0.0005,32,128,3,768,resnet,0.557982,22,1786624,175.051855
48,0.0010,16,128,2,512,resnet,0.579212,12,1456640,202.728135
25,0.0020,16,128,3,512,resnet,0.582269,5,1720576,159.472985
176,0.0005,16,128,2,512,resnet,0.587169,12,1456640,201.232996
365,0.0010,32,128,3,768,resnet,0.587219,26,1786624,202.911120
...,...,...,...,...,...,...,...,...,...,...
151,0.0020,64,512,3,512,transformer,0.677437,29,17112832,157.215956
295,0.0020,64,512,2,1024,transformer,0.677505,8,13171200,55.963006
376,0.0005,16,256,3,512,resnet,0.677691,19,5012992,300.502268
388,0.0010,16,256,2,512,transformer,0.677780,28,4095232,430.410539
